In [25]:
import csv
import requests

In [26]:
params = {
    "part": "snippet",
    "videoId": input("Id do vídeo: "),
    "key": input("Chave de api: "),
    "maxResults": 100
}

In [27]:
next_page_token = None
total_comments = 0

In [28]:
import re

output_file = "youtube_comments.csv"

with open(output_file, mode="w", encoding="utf-8", newline="") as file:
        # Configurar o writer para o CSV
        writer = csv.writer(file, delimiter=";")
        writer.writerow(["author", "message", "like_count", "time"])

        while True:
            if next_page_token:
                params["pageToken"] = next_page_token

            response = requests.get("https://www.googleapis.com/youtube/v3/commentThreads", params=params)
            data = response.json()

            if "items" not in data:
                print("Nenhum comentário encontrado")
                break

            for item in data["items"]:
                comment = item["snippet"]["topLevelComment"]
                author = comment["snippet"]["authorDisplayName"]
                message = comment["snippet"]["textDisplay"]
                like_count = comment["snippet"]["likeCount"]
                time = comment["snippet"]["publishedAt"]

                message = message.replace("\n", " ")
                message = message.replace("\r", " ")
                message = message.replace('"', "'")
                message = message.replace("&quot;", "'")
                message = message.replace(";", ".")
                message = re.sub(r'<[^>]+>', '', message)

                #Escreve no csv
                writer.writerow([author, message, like_count, time])
                total_comments += 1

            next_page_token = data.get("nextPageToken")
            if not next_page_token:
                break